In [1]:
# lib's
import time
from PIL import Image

import numpy as np
import pandas as pd 
import glob
import cv2

import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from skimage import io, color




import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from typing import List, Tuple, Dict
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import regularizers
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.layers import concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.preprocessing import StandardScaler



import os
import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import layers, models, optimizers, regularizers

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage import io, color
from tensorflow.keras.layers import LeakyReLU
from sklearn.preprocessing import MinMaxScaler



import numpy as np
import tensorflow as tf
import matplotlib as mpl
import pandas as pd
import gensim as gs
import sklearn.metrics.pairwise as skpairwise
import gensim.downloader as gdownloader
from matplotlib import pyplot as plt
from typing import List, Tuple, Dict
#from google.colab.patches 
import cv2
import glob


from gensim.models.keyedvectors import KeyedVectors


In [7]:
# variables

counter_rat=0
counter_pig=0
counter_raccoon=0
counter_cat=0
counter_leopard=0
counter_humpback=0
counter_hippopotamus=0
counter_panda=0
counter_chimp=0
counter_seal=0
dists = [[1],[3],[5],[3],[1],[3],[5]]
angles = [[0],[0],[0],[np.pi/4],[np.pi/2],[np.pi/2],[np.pi/2]]
save_pred = {}

# Assuming theta_list and lamda_list are defined
gamma = 0.5
sigma = 0.56
theta_list = [0, np.pi, np.pi/2, np.pi/4, 3*np.pi/4]  # Angles
phi = 0
lamda_list = [2*np.pi/1, 2*np.pi/2, 2*np.pi/3, 2*np.pi/4, 2*np.pi/5]  # Wavelengths


model1 = tf.keras.models.load_model('resnet50_GLCM_Gabor_tamura_ZSL_25_may_2024_model.h5')


# fast_text_vectors.save('fstwk.d2v')
# fast_text_vectors = KeyedVectors.load("fstwk.d2v")

In [15]:
# function's

# Function to calculate Gabor features for an image
def calculate_gabor_features(image):
    image_gray = color.rgb2gray(image)
    image_gray = (image_gray * 255).astype(np.uint8)
    
    local_energy_list = []
    mean_ampl_list = []

    for theta in theta_list:
        for lamda in lamda_list:
            kernel = cv2.getGaborKernel((3, 3), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
            fimage = cv2.filter2D(image_gray, cv2.CV_8UC3, kernel)

            mean_ampl = np.sum(abs(fimage))
            mean_ampl_list.append(mean_ampl)

            local_energy = np.sum(fimage**2)
            local_energy_list.append(local_energy)

#     print("Gabor Mean Amplitude:", mean_ampl_list)
#     print("Gabor Local Energy:", local_energy_list, '\n\n')
    
#     print("return :", mean_ampl_list + local_energy_list, '\n\n')
    
    return np.array(mean_ampl_list + local_energy_list)

# Function to calculate texture features for an image
# def calculate_texture_features(image):
#     image_gray = color.rgb2gray(image)
#     image_gray = (image_gray * 255).astype(np.uint8)
#     df = pd.DataFrame()

#     for n ,(dist, angle) in enumerate(zip(dists, angles)):
        
#         GLCM = greycomatrix(image_gray, dist, angle)
#         GLCM_Energy = greycoprops(GLCM, 'energy')[0]
#         df['Energy'+str(n)] = GLCM_Energy
#         GLCM_corr = greycoprops(GLCM, 'correlation')[0]
#         df['Corr'+str(n)] = GLCM_corr       
#         GLCM_diss = greycoprops(GLCM, 'dissimilarity')[0]
#         df['Diss_sim'+str(n)] = GLCM_diss       
#         GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
#         df['Homogen'+str(n)] = GLCM_hom       
#         GLCM_contr = greycoprops(GLCM, 'contrast')[0]
#         df['Contrast'+str(n)] = GLCM_contr
# #         GLCM_contr = greycoprops(GLCM, 'ASM')[0]
# #         df['ASM'+str(n)] = GLCM_contr

# #     print("Texture Features:", df)
#     return df

def compute_texture_features(image_path):
    # Define your GLCM properties (distances and angles)
    distances = [1, 3, 5, 3, 1, 3, 5]
    angles = [0, 0, 0, np.pi/4, np.pi/2, np.pi/2, np.pi/2]
    
    glcm_features = []
    for distance, angle in zip(distances, angles):
        img_gray = Image.open(image_path).convert('L')  # Convert image to grayscale
        img_gray_array = np.array(img_gray)

        # Compute GLCM properties
        glcm = greycomatrix(img_gray_array, distances=[distance], angles=[angle], levels=256, symmetric=True, normed=True)
        contrast = greycoprops(glcm, 'contrast')[0, 0]
        dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
        homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
        energy = greycoprops(glcm, 'energy')[0, 0]
        correlation = greycoprops(glcm, 'correlation')[0, 0]
        asm = greycoprops(glcm, 'ASM')[0, 0]
        
#         print('contrast, dissimilarity, homogeneity, energy, correlation')
#         print(contrast, dissimilarity, homogeneity, energy, correlation)
        
        glcm_features.extend([contrast, dissimilarity, homogeneity, energy, correlation, asm])
        
    return np.array(glcm_features) # np.array([contrast, dissimilarity, homogeneity, energy, correlation])


def coarseness(image, kmax):
    image = np.array(image)
    w = image.shape[0]
    h = image.shape[1]
    kmax = kmax if (np.power(2, kmax) < w) else int(np.log(w) / np.log(2))
    kmax = kmax if (np.power(2, kmax) < h) else int(np.log(h) / np.log(2))
    average_gray = np.zeros([kmax, w, h])
    horizon = np.zeros([kmax, w, h])
    vertical = np.zeros([kmax, w, h])
    Sbest = np.zeros([w, h])

    for k in range(kmax):
        window = np.power(2, k)
        for wi in range(w)[window:(w - window)]:
            for hi in range(h)[window:(h - window)]:
                average_gray[k][wi][hi] = np.sum(image[wi - window:wi + window, hi - window:hi + window])
        for wi in range(w)[window:(w - window - 1)]:
            for hi in range(h)[window:(h - window - 1)]:
                horizon[k][wi][hi] = average_gray[k][wi + window][hi] - average_gray[k][wi - window][hi]
                vertical[k][wi][hi] = average_gray[k][wi][hi + window] - average_gray[k][wi][hi - window]
        horizon[k] = horizon[k] * (1.0 / np.power(2, 2 * (k + 1)))
        vertical[k] = vertical[k] * (1.0 / np.power(2, 2 * (k + 1)))

    for wi in range(w):
        for hi in range(h):
            h_max = np.max(horizon[:, wi, hi])
            h_max_index = np.argmax(horizon[:, wi, hi])
            v_max = np.max(vertical[:, wi, hi])
            v_max_index = np.argmax(vertical[:, wi, hi])
            index = h_max_index if (h_max > v_max) else v_max_index
            Sbest[wi][hi] = np.power(2, index)

    fcrs = np.mean(Sbest)
    return fcrs

def contrast(image):
    image = np.array(image)
    image = np.reshape(image, (1, image.shape[0] * image.shape[1]))
    m4 = np.mean(np.power(image - np.mean(image), 4))
    v = np.var(image)
    std = np.power(v, 0.5)
    alfa4 = m4 / np.power(v, 2)
    fcon = std / np.power(alfa4, 0.25)
    return fcon

def directionality(image):
    image = np.array(image, dtype='int64')
    h = image.shape[0]
    w = image.shape[1]
    convH = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
    convV = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
    deltaH = np.zeros([h, w])
    deltaV = np.zeros([h, w])
    theta = np.zeros([h, w])

    for hi in range(h)[1:h - 1]:
        for wi in range(w)[1:w - 1]:
            deltaH[hi][wi] = np.sum(np.multiply(image[hi - 1:hi + 2, wi - 1:wi + 2], convH))
    for wi in range(w)[1:w - 1]:
        deltaH[0][wi] = image[0][wi + 1] - image[0][wi]
        deltaH[h - 1][wi] = image[h - 1][wi + 1] - image[h - 1][wi]
    for hi in range(h):
        deltaH[hi][0] = image[hi][1] - image[hi][0]
        deltaH[hi][w - 1] = image[hi][w - 1] - image[hi][w - 2]

    for hi in range(h)[1:h - 1]:
        for wi in range(w)[1:w - 1]:
            deltaV[hi][wi] = np.sum(np.multiply(image[hi - 1:hi + 2, wi - 1:wi + 2], convV))
    for wi in range(w):
        deltaV[0][wi] = image[1][wi] - image[0][wi]
        deltaV[h - 1][wi] = image[h - 1][wi] - image[h - 2][wi]
    for hi in range(h)[1:h - 1]:
        deltaV[hi][0] = image[hi + 1][0] - image[hi][0]
        deltaV[hi][w - 1] = image[hi + 1][w - 1] - image[hi][w - 1]

    deltaG = (np.absolute(deltaH) + np.absolute(deltaV)) / 2.0
    deltaG_vec = np.reshape(deltaG, (deltaG.shape[0] * deltaG.shape[1]))

    for hi in range(h):
        for wi in range(w):
            if (deltaH[hi][wi] == 0 and deltaV[hi][wi] == 0):
                theta[hi][wi] = 0
            elif (deltaH[hi][wi] == 0):
                theta[hi][wi] = np.pi
            else:
                theta[hi][wi] = np.arctan(deltaV[hi][wi] / deltaH[hi][wi]) + np.pi / 2.0
    theta_vec = np.reshape(theta, (theta.shape[0] * theta.shape[1]))

    n = 16
    t = 12
    cnt = 0
    hd = np.zeros(n)
    dlen = deltaG_vec.shape[0]
    for ni in range(n):
        for k in range(dlen):
            if ((deltaG_vec[k] >= t) and (theta_vec[k] >= (2 * ni - 1) * np.pi / (2 * n)) and (
                    theta_vec[k] < (2 * ni + 1) * np.pi / (2 * n))):
                hd[ni] += 1
    hd = hd / np.mean(hd)
    hd_max_index = np.argmax(hd)
    fdir = 0
    for ni in range(n):
        fdir += np.power((ni - hd_max_index), 2) * hd[ni]
    return fdir

def roughness(fcrs, fcon):
    return fcrs + fcon


In [25]:
# main

t0 = time.time()

texture_features, gabor_features, tamura_features = [], [], []

for img_path in glob.glob(r"data1_mini\*\*"):
    target_size = (224, 224)        
    #target_size = (224, 224)  # Specify the desired target size
    
    #image1 = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)

    image = Image.open(img_path)
#     print('image type : ', type(image))
    
    # Convert the PIL image to a NumPy array
    image_array = np.array(image)

    
    # Preprocess the image
    preprocessed_img = tf.keras.applications.vgg19.preprocess_input(tf.image.resize(image_array, (224, 224)))

    # Expand dimensions of the image to match model input shape
    preprocessed_img = np.expand_dims(preprocessed_img, axis=0)

#     Calculate Gabor features for the test image
    gabor_feature = calculate_gabor_features(image_array)
    gabor_features.append(gabor_feature)
    print(f"gabor_features : {gabor_features}")

    # Normalize the Gabor features
    scaler_gabor = StandardScaler()
#     gabor_features_normalized = scaler_gabor.fit_transform(np.array(gabor_features).reshape(1, -1))
    gabor_features_normalized = scaler_gabor.fit_transform(gabor_features)

    
    
#     # Calculate texture features for the test image
#     texture_features_df = calculate_texture_features(image_array)
#     scaler_texture = StandardScaler()
#     texture_features_normalized = scaler_texture.fit_transform(texture_features_df)
    # Compute texture features using GLCM
    texture_feat = compute_texture_features(img_path)
    texture_features.append(texture_feat)
    print(f"texture_features : {texture_features}")
    
    scaler_texture = StandardScaler()
    texture_features_normalized = scaler_texture.fit_transform(texture_features)


    # Calculate tamura features for the test image
    image_gray = color.rgb2gray(image_array)
    image_gray = (image_gray * 255).astype(np.uint8)
    fcrs = coarseness(image_gray, 5)
    fcon = contrast(image_gray)
    fdir = directionality(image_gray)
    f_r = roughness(fcrs, fcon)
    tamura_feature = np.array([fcrs, fcon, fdir, f_r])
    tamura_features.append(tamura_feature)
    print(f"tamura_features : {tamura_features}")
    
    
#     scaler_tamura = StandardScaler()
#     tamura_features_normalized = scaler_tamura.fit_transform(tamura_features.reshape(1, -1))
    
    scaler_tamura = StandardScaler()
    tamura_features_normalized = scaler_tamura.fit_transform(tamura_features)


#     combined_features = np.hstack((gabor_features_normalized, texture_features_normalized, tamura_features_normalized))

#     combined_features_df = pd.DataFrame(combined_features, columns=[
#         f'Mean_Ampl_{i + 1}' for i in range(len(theta_list) * len(lamda_list))
#     ] + [
#         f'Local_Energy_{i + 1}' for i in range(len(theta_list) * len(lamda_list))
#     ] + texture_features_df.columns.tolist() + [
#         'Coarseness', 'Contrast', 'Directionality', 'Roughness'
#     ])
    
    combined_features_df = pd.DataFrame(np.concatenate([gabor_features, texture_features, tamura_features], axis=1))
    print('combined_features_df : \n\n', combined_features_df)

        
#     predictions = model1.predict([combined_features_df, preprocessed_img], verbose=1)

#     print('predictions : ', predictions[0])
    
    
    
    
    
    # get top-n labels by cosine similarity
#     most_similar: List[str] = fast_text_vectors.most_similar(
#         predictions[0], topn=15)
#    # most_similar: List[str] = fast_text_vectors.similar_by_vector(
#     #    predictions[0], topn=10)
#     # display image
#     plt.figure(figsize=(2,2))
#     plt.imshow(image1)
#     plt.axis('off')
#     plt.show()
    
#     for i in most_similar:
#                 if i[0] == 'walrus':
#                     counter_seal+=1
      
#     for i in most_similar:
#                 if i[0] == 'chimp':
#                     counter_chimp+=1
                    
#     for i in most_similar:
#                 if i[0] == 'bear':
#                     counter_panda+=1
                    
#     for i in most_similar:
#                 if i[0] == 'hippopotamus':
#                     counter_hippopotamus+=1
    
#     for i in most_similar:
#                 if i[0] == 'humpback':
#                     counter_humpback+=1
                    
#     for i in most_similar:
#                 if i[0] == img_path.split("\\")[-2]:
#                       counter_leopard+=1
            
#     for i in most_similar:
#                 if i[0] == img_path.split("\\")[-2]:
#                       counter_raccoon+=1
    
#     for i in most_similar:
#                 if i[0] == 'cat':
#                     counter_cat+=1
     
    
#     for i in most_similar:
#                 if i[0] == img_path.split("\\")[-2]:
#                       counter_pig+=1
                    
#     for i in most_similar:
#                 if i[0] == img_path.split("\\")[-2]:
#                     counter_rat+=1
                    
#     # print the predictions for image
#     save_pred[img_path] = [x[0] for x in most_similar]
#     print("foler",img_path)
#     print(f"Prediction for image: {', '.join([x[0] for x in most_similar])}")
    
    break
    
print("Training time:", time.time()-t0)


gabor_features : [array([165881110, 147546407, 121721726, 139158873, 162295454, 165881110,
       147546407, 121721726, 139158873, 162295454, 165862585, 147725517,
       121587452, 139284081, 162306849, 168339072, 151111054,  98029461,
        50615860,  53548297, 168282955, 150985914,  98011377,  50623963,
        53554903,  42553102,  59853789,  85645798,  69206947,  44803976,
        42553102,  59853789,  85645798,  69206947,  44803976,  42375215,
        59779681,  86365990,  69827747,  44599465,  40609966,  56506036,
        93727823,  96787272,  95264507,  40618679,  56365494,  93973225,
        97103011,  95470227], dtype=uint32)]
texture_features : [array([1.55988680e+02, 6.95092841e+00, 2.97337623e-01, 3.01197021e-02,
       9.82571968e-01, 9.07196455e-04, 3.95501271e+02, 1.09299695e+01,
       1.99994381e-01, 2.24963828e-02, 9.55786361e-01, 5.06087239e-04,
       5.86600629e+02, 1.30561360e+01, 1.62679647e-01, 1.93442129e-02,
       9.34384561e-01, 3.74198571e-04, 3.71259943

C:\Project_Work\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
C:\Project_Work\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\Project_Work\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\Project_Work\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be 

tamura_features : [array([  9.09888713,  57.81342308, 257.4319129 ,  66.9123102 ])]
combined_features_df : 

             0            1            2            3            4   \
0  165881110.0  147546407.0  121721726.0  139158873.0  162295454.0   

            5            6            7            8            9   ...  \
0  165881110.0  147546407.0  121721726.0  139158873.0  162295454.0  ...   

          86         87        88        89        90        91        92  \
0  602.37006  12.700309  0.179441  0.020424  0.932611  0.000417  9.098887   

          93          94        95  
0  57.813423  257.431913  66.91231  

[1 rows x 96 columns]
Training time: 80.29802823066711


In [24]:
# combined_features = np.hstack((gabor_features_normalized, texture_features_normalized, tamura_features_normalized))

print('combined_features : ', gabor_features, texture_features, tamura_features)

pd.DataFrame(np.concatenate([gabor_features, texture_features, tamura_features], axis=1))

# combined_features_df = pd.DataFrame(combined_features, columns=[
#     f'Mean_Ampl_{i + 1}' for i in range(len(theta_list) * len(lamda_list))
# ] + [
#     f'Local_Energy_{i + 1}' for i in range(len(theta_list) * len(lamda_list))
# ] + texture_features_df.columns.tolist() + [
#     'Coarseness', 'Contrast', 'Directionality', 'Roughness'
# ])

# print('combined_features_df : ', combined_features_df)

combined_features :  [array([165881110, 147546407, 121721726, 139158873, 162295454, 165881110,
       147546407, 121721726, 139158873, 162295454, 165862585, 147725517,
       121587452, 139284081, 162306849, 168339072, 151111054,  98029461,
        50615860,  53548297, 168282955, 150985914,  98011377,  50623963,
        53554903,  42553102,  59853789,  85645798,  69206947,  44803976,
        42553102,  59853789,  85645798,  69206947,  44803976,  42375215,
        59779681,  86365990,  69827747,  44599465,  40609966,  56506036,
        93727823,  96787272,  95264507,  40618679,  56365494,  93973225,
        97103011,  95470227], dtype=uint32)] [array([1.55988680e+02, 6.95092841e+00, 2.97337623e-01, 3.01197021e-02,
       9.82571968e-01, 9.07196455e-04, 3.95501271e+02, 1.09299695e+01,
       1.99994381e-01, 2.24963828e-02, 9.55786361e-01, 5.06087239e-04,
       5.86600629e+02, 1.30561360e+01, 1.62679647e-01, 1.93442129e-02,
       9.34384561e-01, 3.74198571e-04, 3.71259943e+02, 1.0535156

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,165881110.0,147546407.0,121721726.0,139158873.0,162295454.0,165881110.0,147546407.0,121721726.0,139158873.0,162295454.0,...,602.37006,12.700309,0.179441,0.020424,0.932611,0.000417,9.098887,57.813423,257.431913,66.91231
